In [644]:
import pandas as pd

In [645]:
# Reading the csv file and storing in to TimeSheet dataframe

In [646]:
TimeSheet = pd.read_csv("D:/Neo/Experiments/TimeSheet.csv") 

In [647]:
# Extracting the development effort by identifying the stories

In [648]:
story=TimeSheet["Issue Type"] == "Story"  

In [649]:
newgoe=TimeSheet["Reporter"] == "vinotha.s"

In [650]:
goedev=TimeSheet["Time Spent"] > 0

In [651]:
dev_efforts=TimeSheet[story & newgoe & goedev].groupby('Issue id')['Time Spent'].sum().reset_index(name ='Dev_Hours')

In [652]:
dev_efforts["Dev_Hours"] = dev_efforts["Dev_Hours"]/3600

In [653]:
# Extracting the sub task efforts by identifying the sub tasks

In [654]:
subtasks=TimeSheet["Issue Type"] == "Sub-task"

In [655]:
efforts_st=TimeSheet[subtasks]

In [656]:
# Seggregating the different types of sub tasks

In [657]:
efforts_ds=efforts_st["Summary"].str.contains("Design")

In [658]:
efforts_bfix=efforts_st["Summary"].str.contains("Fixing")

In [659]:
efforts_ret=efforts_st["Summary"].str.contains("Retesting")

In [660]:
efforts_tes=efforts_st["Summary"].str.contains("Testing")

In [661]:
# Calculating the design effort

In [662]:
des_efforts=efforts_st[efforts_ds].groupby('Parent id')['Time Spent'].sum().reset_index(name ='Design_Hours')

In [663]:
des_efforts["Parent id"]= des_efforts["Parent id"].astype(int)

In [664]:
des_efforts["Design_Hours"]= des_efforts["Design_Hours"]/3600

In [665]:
# Calculating the bug fix effort

In [666]:
bugfix_efforts=efforts_st[efforts_bfix].groupby('Parent id')['Time Spent'].sum().reset_index(name ='BugFix_Hours')


In [667]:
bugfix_efforts["Parent id"]= bugfix_efforts["Parent id"].astype(int)


In [668]:
bugfix_efforts["BugFix_Hours"]= round(bugfix_efforts["BugFix_Hours"]/3600,2)


In [669]:
# Calculating the bug retesting effort

In [670]:
bugret_efforts=efforts_st[efforts_ret].groupby('Parent id')['Time Spent'].sum().reset_index(name ='BugRetest_Hours')

In [671]:
bugret_efforts["Parent id"]= bugret_efforts["Parent id"].astype(int)

In [672]:
bugret_efforts["BugRetest_Hours"]= bugret_efforts["BugRetest_Hours"]/3600

In [673]:
# Calculating the testing effort

In [674]:
Test_efforts=efforts_st[efforts_tes].groupby('Parent id')['Time Spent'].sum().reset_index(name ='Test_Hours')

In [675]:
Test_efforts["Parent id"]= Test_efforts["Parent id"].astype(int)

In [676]:
Test_efforts["Test_Hours"]= Test_efforts["Test_Hours"]/3600

In [677]:
# Merging the design, testing, restesting and bug fixing efforts to subtaskeffort

In [678]:
subtaskeffort = pd.merge(des_efforts,Test_efforts, on="Parent id",how="outer")

In [679]:
subtaskeffort = pd.merge(subtaskeffort,bugfix_efforts, on="Parent id",how="outer")

In [680]:
subtaskeffort = pd.merge(subtaskeffort,bugret_efforts, on="Parent id",how="outer")

In [681]:
# Renaming the columns to "Id" for dev effort and for the sub task effort

In [682]:
dev_efforts=dev_efforts.rename(index=str,columns = {"Issue id":"Id"})

In [683]:
subtaskeffort= subtaskeffort.rename(index=str,columns = {"Parent id":"Id"})

In [684]:
# Extracting the data issue id and issue key to a new dataframe to combine with the rest and also rename the column to "id"

In [685]:
ticket_id=TimeSheet[story & newgoe & goedev].filter(['Issue id','Issue key'], axis=1)

In [686]:
ticket_id=ticket_id.rename(index=str,columns = {"Issue id":"Id"})

In [687]:
storyeffort=pd.merge(dev_efforts,subtaskeffort,on="Id",how="outer")

In [688]:
storyeffort=pd.merge(storyeffort,ticket_id,on="Id",how="outer")

In [689]:
storyeffort=storyeffort.fillna(0)

In [690]:
col_list= list(storyeffort)
col_list.remove('Id')
col_list.remove('Issue key')

In [691]:
storyeffort["Total Hours"] = storyeffort[col_list].sum(axis=1)

In [692]:
storyeffort

,Id,Dev_Hours,Design_Hours,Test_Hours,BugFix_Hours,BugRetest_Hours,Issue key,Total Hours
0,36423,8.00,0.00,0.0,0.00,0.0,ENBD-172,8.00
1,36559,76.25,5.50,6.5,60.50,10.0,ENBD-176,158.75
2,36562,7.25,33.50,14.0,9.00,0.0,ENBD-177,63.75
3,36563,114.75,0.00,0.0,0.00,0.0,ENBD-178,114.75
4,36565,8.00,7.75,8.0,46.17,4.0,ENBD-180,73.92
5,36574,2.00,36.50,0.0,21.00,5.5,ENBD-185,65.00
6,36578,71.00,5.00,10.0,0.00,2.5,ENBD-186,88.50
7,36579,72.50,4.00,0.0,26.75,0.0,ENBD-187,103.25
8,36580,49.50,1.00,6.5,30.00,0.0,ENBD-188,87.00
9,36592,35.00,0.00,0.0,1.00,1.5,ENBD-189,37.50
